In [1]:
import pandas as pd

# Read data from files 
train = pd.read_csv( "labeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )
test = pd.read_csv( "testData.tsv", header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv( "unlabeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )

# Verify the number of reviews that were read (100,000 in total)
print ("Read %d labeled train reviews, %d labeled test reviews, " \
 "and %d unlabeled reviews\n" % (train["review"].size,  
 test["review"].size, unlabeled_train["review"].size ))

Read 25000 labeled train reviews, 25000 labeled test reviews, and 50000 unlabeled reviews



The functions we write to clean the data are also similar to Part 1, although now there are a couple of differences. First, to train Word2Vec it is better not to remove stop words because the algorithm relies on the broader context of the sentence in order to produce high-quality word vectors. For this reason, we will make stop word removal optional in the functions below. It also might be better not to remove numbers, but we leave that as an exercise for the reader.

In [2]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

Next, we want a specific input format. Word2Vec expects single sentences, each one as a list of words. In other words, the input format is a list of lists.

It is not at all straightforward how to split a paragraph into sentences. There are all kinds of gotchas in natural language. English sentences can end with "?", "!", """, or ".", among other things, and spacing and capitalization are not reliable guides either. For this reason, we'll use NLTK's punkt tokenizer for sentence splitting. In order to use this, you will need to install NLTK and use nltk.download() to download the relevant training file for punkt.

In [3]:
# Download the punkt tokenizer for sentence splitting
import nltk.data
# nltk.download()   

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

Now we can apply this function to prepare our data for input to Word2Vec (this will take a couple minutes):

In [5]:
sentences = []  # Initialize an empty list of sentences

print ("Parsing sentences from training set")
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

print ("Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)


Parsing sentences from training set


C:\Users\saurabh\Anaconda3\lib\site-packages\bs4\__init__.py:272: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\Users\saurabh\Anaconda3\lib\site-packages\bs4\__init__.py:272: UserWarning: "b'...'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\Users\saurabh\Anaconda3\lib\site-packages\bs4\__init__.py:335: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


Parsing sentences from unlabeled set


C:\Users\saurabh\Anaconda3\lib\site-packages\bs4\__init__.py:335: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\saurabh\Anaconda3\lib\site-packages\bs4\__init__.py:335: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\saurabh\Anaconda3\lib\site-packages\bs4\__init__.py:272: UserWarning: "b'... ...'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\Users\sa

You may get a few warnings from BeautifulSoup about URLs in the sentences. These are nothing to worry about (although you may want to consider removing URLs when cleaning the text). 

We can take a look at the output to see how this differs from Part 1:

In [6]:
print(len(sentences))

795538


In [7]:
print (sentences[0])

['with', 'all', 'this', 'stuff', 'going', 'down', 'at', 'the', 'moment', 'with', 'mj', 'i', 've', 'started', 'listening', 'to', 'his', 'music', 'watching', 'the', 'odd', 'documentary', 'here', 'and', 'there', 'watched', 'the', 'wiz', 'and', 'watched', 'moonwalker', 'again']


Training and Saving Your Model
With the list of nicely parsed sentences, we're ready to train the model. There are a number of parameter choices that affect the run time and the quality of the final model that is produced. For details on the algorithms below, see the word2vec API documentation as well as the Google documentation. 

Architecture: Architecture options are skip-gram (default) or continuous bag of words. We found that skip-gram was very slightly slower but produced better results.

Training algorithm: Hierarchical softmax (default) or negative sampling. For us, the default worked well.

Downsampling of frequent words: The Google documentation recommends values between .00001 and .001. For us, values closer 0.001 seemed to improve the accuracy of the final model.

Word vector dimensionality: More features result in longer runtimes, and often, but not always, result in better models. Reasonable values can be in the tens to hundreds; we used 300.

Context / window size: How many words of context should the training algorithm take into account? 10 seems to work well for hierarchical softmax (more is better, up to a point).

Worker threads: Number of parallel processes to run. This is computer-specific, but between 4 and 6 should work on most systems.

Minimum word count: This helps limit the size of the vocabulary to meaningful words. Any word that does not occur at least this many times across all documents is ignored. Reasonable values could be between 10 and 100. In this case, since each movie occurs 30 times, we set the minimum word count to 40, to avoid attaching too much importance to individual movie titles. This resulted in an overall vocabulary size of around 15,000 words. Higher values also help limit run time.
Choosing parameters is not easy, but once we have chosen our parameters, creating a Word2Vec model is straightforward:

In [8]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print ("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

2019-11-05 03:33:21,817 : INFO : 'pattern' package not found; tag filters are not available for English
C:\Users\saurabh\Anaconda3\lib\site-packages\gensim\models\base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "
2019-11-05 03:33:21,895 : INFO : collecting all words and their counts
2019-11-05 03:33:21,900 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-11-05 03:33:22,064 : INFO : PROGRESS: at sentence #10000, processed 225803 words, keeping 17776 word types


Training model...


2019-11-05 03:33:22,236 : INFO : PROGRESS: at sentence #20000, processed 451892 words, keeping 24948 word types
2019-11-05 03:33:22,482 : INFO : PROGRESS: at sentence #30000, processed 671315 words, keeping 30034 word types
2019-11-05 03:33:22,640 : INFO : PROGRESS: at sentence #40000, processed 897815 words, keeping 34348 word types
2019-11-05 03:33:22,799 : INFO : PROGRESS: at sentence #50000, processed 1116963 words, keeping 37761 word types
2019-11-05 03:33:22,982 : INFO : PROGRESS: at sentence #60000, processed 1338404 words, keeping 40723 word types
2019-11-05 03:33:23,115 : INFO : PROGRESS: at sentence #70000, processed 1561580 words, keeping 43333 word types
2019-11-05 03:33:23,266 : INFO : PROGRESS: at sentence #80000, processed 1780887 words, keeping 45714 word types
2019-11-05 03:33:23,486 : INFO : PROGRESS: at sentence #90000, processed 2004996 words, keeping 48135 word types
2019-11-05 03:33:23,859 : INFO : PROGRESS: at sentence #100000, processed 2226966 words, keeping 50

2019-11-05 03:33:42,732 : INFO : PROGRESS: at sentence #740000, processed 16552903 words, keeping 119668 word types
2019-11-05 03:33:42,894 : INFO : PROGRESS: at sentence #750000, processed 16771230 words, keeping 120295 word types
2019-11-05 03:33:43,061 : INFO : PROGRESS: at sentence #760000, processed 16990622 words, keeping 120930 word types
2019-11-05 03:33:43,221 : INFO : PROGRESS: at sentence #770000, processed 17217759 words, keeping 121703 word types
2019-11-05 03:33:43,381 : INFO : PROGRESS: at sentence #780000, processed 17447905 words, keeping 122402 word types
2019-11-05 03:33:43,524 : INFO : PROGRESS: at sentence #790000, processed 17674981 words, keeping 123066 word types
2019-11-05 03:33:43,614 : INFO : collected 123504 word types from a corpus of 17798082 raw words and 795538 sentences
2019-11-05 03:33:43,616 : INFO : Loading a fresh vocabulary
2019-11-05 03:33:44,141 : INFO : effective_min_count=40 retains 16490 unique words (13% of original 123504, drops 107014)
2019

2019-11-05 03:43:58,470 : INFO : EPOCH 1 - PROGRESS: at 4.54% examples, 946 words/s, in_qsize 8, out_qsize 0
2019-11-05 03:44:04,855 : INFO : EPOCH 1 - PROGRESS: at 4.59% examples, 948 words/s, in_qsize 7, out_qsize 0
2019-11-05 03:44:08,566 : INFO : EPOCH 1 - PROGRESS: at 4.70% examples, 965 words/s, in_qsize 8, out_qsize 0
2019-11-05 03:44:27,304 : INFO : EPOCH 1 - PROGRESS: at 4.75% examples, 948 words/s, in_qsize 7, out_qsize 0
2019-11-05 03:44:34,433 : INFO : EPOCH 1 - PROGRESS: at 4.81% examples, 949 words/s, in_qsize 7, out_qsize 0
2019-11-05 03:44:37,886 : INFO : EPOCH 1 - PROGRESS: at 4.92% examples, 966 words/s, in_qsize 7, out_qsize 0
2019-11-05 03:44:57,667 : INFO : EPOCH 1 - PROGRESS: at 4.98% examples, 948 words/s, in_qsize 8, out_qsize 0
2019-11-05 03:45:04,928 : INFO : EPOCH 1 - PROGRESS: at 5.03% examples, 948 words/s, in_qsize 7, out_qsize 0
2019-11-05 03:45:07,397 : INFO : EPOCH 1 - PROGRESS: at 5.14% examples, 966 words/s, in_qsize 8, out_qsize 0
2019-11-05 03:45:26

KeyboardInterrupt: 

On a dual-core Macbook Pro, this took less than 15 minutes to run using 4 worker threads. However, it will vary depending on your computer. Fortunately, the logging functionality prints informative messages.

If you are on a Mac or Linux system, you can use the "top" command from within Terminal (not from within Python) to see if your system is successfully parallelizing while the model is training. Type

In [ ]:
model.doesnt_match("man woman child kitchen".split())

Our model is capable of distinguishing differences in meaning! It knows that men, women and children are more similar to each other than they are to kitchens. More exploration shows that the model is sensitive to more subtle differences in meaning, such as differences between countries and cities

In [ ]:
model.doesnt_match("france england germany berlin".split())
'berlin'

In [ ]:
model.doesnt_match("paris berlin london austria".split())
'paris'

In [ ]:
model.most_similar("man")

In [ ]:
model.most_similar("queen")

In [ ]:
model.most_similar("awful")